In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import os
from sklearn.metrics import f1_score
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from datetime import datetime
from xgboost import XGBClassifier
from numpy import inf

C:\Users\omore\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# Путь к директории с данными

data_dir = '../../00_data/track_1/'
sumbit_dir = '../../02_submits/'
preprocessed_dir = '../../03_preprocessed_data/'

X_train = pd.read_csv(preprocessed_dir + 'X_train.csv')
X_test = pd.read_csv(preprocessed_dir + 'X_test.csv')
y_train = pd.read_csv(preprocessed_dir + 'y_train.csv')
y_test  = pd.read_csv(preprocessed_dir + 'y_test.csv')
X = pd.read_csv(preprocessed_dir + 'X.csv')
unused_features = [ 'is_test']
cat_col = [x for x in X_train.columns if 'code'  in x]
unused_features = unused_features + cat_col
use_for_train = ['year', 'station_id', 'day', 'ice_jam_same_day_last_year',
       'ice_jam_same_day_last_year_prev_stat', 'ice_jam_same_day_prelast_year',
       'is_prelast_year_missing', 'n_jams_last_year',
       'n_jams_prelast_year', 'n_jams_last_year_prev_station', 'n_jams_past',
       'n_jams_past_prev_stat', 'n_same_days_before',
       'n_ice_jams_same_days_before', 'n_ice_jams_same_days_before_ratio', 'station_width']
#
use_for_train = ['year', 'station_id', 'day', 
       'ice_jam_same_day_last_year', 
       'ice_jam_same_day_prelast_year', 
                 'is_last_year_missing',
       'is_prelast_year_missing', 
                 'n_jams_last_year',
                 'n_jams_prelast_year',
       'n_jams_past',  'n_ice_jams_same_days_before',
       'n_ice_jams_same_days_before_ratio',
       'distance_from_source', 'drainage_area', 'z_null', 
       'height_diff_by_dist']
use_for_train = use_for_train + \
    [x for x in X_train.columns if 'stage_max'  in x] + \
    [x for x in X_train.columns if 'air'  in x] + \
    [x for x in X_train.columns if 'inc'  in x] 
use_for_train = list(set(use_for_train))

C:\Users\omore\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (45,64,120,139,195,420,589,645,664) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\omore\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (45,64,120,139,195,270,589,645,664) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\omore\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (46,65,121,140,196,271,421,590,646,665) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
len(use_for_train)

97

from sklearn.feature_selection import SelectKBest
N_features = 50
unused_features = unused_features + cat_col
feature_selector = SelectKBest(k = N_features)
features = X_train.drop(unused_features, axis = 1).columns
feature_selector = feature_selector.fit(X_train.drop(unused_features, axis = 1), y_train)
#X_train_new = .fit_transform(X_train.drop(unused_features, axis = 1), y_train)
#X_train_new.shape
use_for_train = features[feature_selector.get_support()]
use_for_train

In [34]:
use_for_train = use_for_train + \
    [x for x in X_train.columns if 'stage_max'  in x] + \
    [x for x in X_train.columns if 'air'  in x] + \
    [x for x in X_train.columns if 'inc'  in x] 
use_for_train = list(set(use_for_train))

In [53]:
model = XGBClassifier(scale_pos_weight = 50, n_estimators = 500, random_state=0)
eval_set =[(X_test_used, y_test)]
model.fit(X_train_used, y_train, early_stopping_rounds = 2, 
                      eval_metric="error",
                     eval_set=eval_set , verbose=True)   

y_pred = model.predict(X_test_used)
y_train_pred = model.predict(X_train_used)

print('F1-train: {}'.format(f1_score(y_train, y_train_pred)))
print('F1-test: {}'.format(f1_score(y_test, y_pred)))

[0]	validation_0-error:0.16436
Will train until validation_0-error hasn't improved in 2 rounds.
[1]	validation_0-error:0.09285
[2]	validation_0-error:0.09950
[3]	validation_0-error:0.07844
[4]	validation_0-error:0.08121
[5]	validation_0-error:0.05543
[6]	validation_0-error:0.05903
[7]	validation_0-error:0.04490
[8]	validation_0-error:0.04130
[9]	validation_0-error:0.03908
[10]	validation_0-error:0.03437
[11]	validation_0-error:0.03326
[12]	validation_0-error:0.02605
[13]	validation_0-error:0.02051
[14]	validation_0-error:0.02411
[15]	validation_0-error:0.02051
Stopping. Best iteration:
[13]	validation_0-error:0.02051

F1-train: 0.3003952569169961
F1-test: 0.075


## Retrain the model on all data

In [75]:
X_used = X[X_test_used.columns.to_list() + ['is_test', 'ice_jam']  ]
all_train = X_used[(X_used['is_test'] == False)].drop('ice_jam', axis = 1).reset_index(drop=True)
X_val = X_used[(X_used['is_test'] == True)].drop('ice_jam', axis = 1).reset_index(drop=True).dropna()
all_y_train =  X_used.loc[(X_used['is_test'] == False),'ice_jam']
unused_features = ['year', 'is_test']
mod_opt= model.fit(all_train, all_y_train)


In [77]:
X_val['ice_jam'] = mod_opt.predict(X_val)

### Save submit file

In [78]:
now = datetime.now()
X_val[['year', 'station_id', 'day', 'ice_jam']].\
    to_csv(sumbit_dir + 'submit-rf' + now.strftime("%m-%d-%Y-%H-%M") + '.csv', index=False)